In [1]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
from sqlalchemy import text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import numpy as np



In [2]:
credentials = "Credentials.json"

#define the function used to connect with pgadmin
def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn
    
    
#define the query function
def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result


#define the WKT function
def create_wkt_element(geom):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)


#Connect to the database
db, conn = pgconnect(credentials)

Connected successfully.


In [3]:
#Task 1 Load the data

In [4]:
#Load the sa2 data

srid = 4326
sa_2 = gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
sa_2 = sa_2[sa_2['GCC_NAME21'] == 'Greater Sydney']
sa_2['geometry'] = sa_2['geometry'].apply(create_wkt_element)


/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/823317905.py:6: UserWarning: Geometry column does not contain geometry.
  sa_2['geometry'] = sa_2['geometry'].apply(create_wkt_element)


In [5]:
#Load the sa2 data
conn.execute(text("""
DROP TABLE IF EXISTS sa2;
CREATE TABLE sa2(
    "SA2_CODE21" INTEGER,
    "SA2_NAME21" VARCHAR(255),
    "CHG_FLAG21" INTEGER,
    "CHG_LBL21" VARCHAR(255),
    "SA3_CODE21" INTEGER,
    "SA3_NAME21" VARCHAR(255),
    "SA4_CODE21" INTEGER,
    "SA4_NAME21" VARCHAR(255),
    "GCC_CODE21" VARCHAR(255),
    "GCC_NAME21" VARCHAR(255),
    "STE_CODE21" INTEGER,
    "STE_NAME21" VARCHAR(255),
    "AUS_CODE21" VARCHAR(255),
    "AUS_NAME21" VARCHAR(255),
    "AREASQKM21" FLOAT,
    "LOCI_URI21" VARCHAR(255),
    "geometry" public.GEOMETRY(MULTIPOLYGON,4326)
);
"""))
sa_2.to_sql("sa2", conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid)})

# Create indexes for the sa2 table
conn.execute("""
CREATE INDEX IF NOT EXISTS sa2_code ON sa2("SA2_CODE21");
CREATE INDEX IF NOT EXISTS sa2_name ON sa2("SA2_NAME21");
""")

query(conn, "SELECT * FROM sa2;")

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000200000001030000000100000085...
3,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000007E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000046...
369,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E61000000100000001030000000100000089...
370,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E6100000010000000103000000010000008E...
371,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020E610000001000000010300000001000000A1...


In [6]:
#Load the business data
business_data = pd.read_csv('Businesses.csv')
business_data = business_data.dropna()
business_data.columns = map(str.lower, business_data.columns)

conn.execute(text("""
DROP TABLE IF EXISTS Businesses;
CREATE TABLE Businesses(
    Industry_code VARCHAR(255),
    Industry_name VARCHAR(255),
    sa2_code INTEGER,
    sa2_name VARCHAR(255),
    "0_to_50k_businesses" INTEGER,
    "50k_to_200k_businesses" INTEGER,
    "200k_to_2m_businesses" INTEGER,
    "2m_to_5m_businesses" INTEGER,
    "5m_to_10m_businesses" INTEGER,
    "10m_or_more_businesses" INTEGER,
    total_businesses INTEGER
);
"""))

business_data.to_sql("businesses", con=conn, if_exists='append', index=False)

# Create indexes for the businesses table
conn.execute(text("""CREATE INDEX IF NOT EXISTS business_sa2_code ON Businesses("sa2_code");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS business_total_businesses ON Businesses("total_businesses");"""))

query(conn, "SELECT * FROM Businesses;")

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
12215,S,Other Services,128021609,Woronora Heights,0,3,5,0,0,0,9


In [7]:
#Load the Population data
population_data=pd.read_csv("Population.csv")
population_data = population_data.dropna()
population_data.columns = map(str.lower, population_data.columns)

conn.execute(text("""
DROP TABLE IF EXISTS Population;
CREATE TABLE Population (
    sa2_code INTEGER,
    sa2_name VARCHAR(50),
    "0-4_people" INTEGER,
    "5-9_people" INTEGER,
    "10-14_people" INTEGER,
    "15-19_people" INTEGER,
    "20-24_people" INTEGER,
    "25-29_people" INTEGER,
    "30-34_people" INTEGER,
    "35-39_people" INTEGER,
    "40-44_people" INTEGER,
    "45-49_people" INTEGER,
    "50-54_people" INTEGER,
    "55-59_people" INTEGER,
    "60-64_people" INTEGER,
    "65-69_people" INTEGER,
    "70-74_people" INTEGER,
    "75-79_people" INTEGER,
    "80-84_people" INTEGER,
    "85-and-over_people" INTEGER,
    total_people INTEGER
);
"""))

population_data.to_sql("population", con=conn, if_exists='append', index=False)

# Create indexes for the population table
conn.execute(text("""CREATE INDEX IF NOT EXISTS population_sa2_code ON Population("sa2_code");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS population_sa2_name ON Population("sa2_name");"""))

query(conn, "SELECT * FROM Population;")

,sa2_code,sa2_name,0-4_people,5-9_people,10-14_people,15-19_people,20-24_people,25-29_people,30-34_people,35-39_people,...,45-49_people,50-54_people,55-59_people,60-64_people,65-69_people,70-74_people,75-79_people,80-84_people,85-and-over_people,total_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,572,602,570,520,464,369,226,142,70,7530
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,749,794,895,863,925,603,331,264,11052
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,325,436,422,397,327,264,190,100,75,4748
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,859,882,901,930,917,1065,976,773,1028,14803
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1330,1241,1377,1285,1166,949,664,476,537,21346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2,4,10,4,9,7,1,2,...,4,0,0,0,1,0,0,0,0,45
369,128021538,Sutherland - Kirrawee,1519,1292,1150,1117,1335,1852,2120,1945,...,1569,1391,1285,1157,909,909,781,582,807,23369
370,128021607,Engadine,1157,1283,1469,1209,891,675,928,1229,...,1315,1086,909,764,707,886,748,389,327,17379
371,128021608,Loftus - Yarrawarrah,503,487,575,508,380,293,426,493,...,564,477,450,387,418,335,263,192,109,7354


In [8]:
#Load the income data
income_data = pd.read_csv("Income.csv")
income_data = income_data.dropna()

#Cleaning the data at first due to there are some regions' data which are unknown
for x in income_data.index:
    if income_data.loc[x, "earners"] == "np":
        income_data.loc[x, "earners"] = -1
    if income_data.loc[x, "median_age"] == "np":
        income_data.loc[x, "median_age"] = -1
    if income_data.loc[x, "median_income"] == "np":
        income_data.loc[x, "median_income"] = -1
    if income_data.loc[x, "mean_income"] == "np":
        income_data.loc[x, "mean_income"] = -1

income_data.columns = map(str.lower, income_data.columns)

conn.execute(text("""
DROP TABLE IF EXISTS Income;
CREATE TABLE Income(
    sa2_code21 INTEGER,
    sa2_name VARCHAR(50),
    earners INTEGER,
    median_age INTEGER,
    median_income INTEGER,
    mean_income INTEGER
);
"""))

income_data.to_sql("income", con=conn, if_exists='append', index=False)

# Create indexes for the income table
conn.execute(text("""CREATE INDEX idx_income_sa2_code21 ON income("sa2_code21");"""))

query(conn, "SELECT * FROM Income;")

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981
...,...,...,...,...,...,...
637,128021537,Royal National Park,14,37,36980,47584
638,128021538,Sutherland - Kirrawee,13895,41,64940,74867
639,128021607,Engadine,10239,43,63695,72995
640,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


In [9]:
#Load stop data
stops_data = pd.read_csv("Stops.txt", quotechar='"')

#create a new column to store the point geometry data
stops_data['geometry'] = stops_data.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)

#convert the dataframe to GeoDataFrame
stops_data = gpd.GeoDataFrame(stops_data, geometry='geometry')

def create_point_wkt_element(geom):
    return WKTElement(geom.wkt, srid)

stops_data['geometry'] = stops_data['geometry'].apply(create_point_wkt_element)

conn.execute(text("""
DROP TABLE IF EXISTS Stops;
CREATE TABLE Stops(
    stop_id VARCHAR(255),
    stop_code VARCHAR(255),
    stop_name VARCHAR(255),
    stop_lat VARCHAR(255),
    stop_lon VARCHAR(255),
    location_type VARCHAR(255),
    parent_station VARCHAR(255),
    wheelchair_boarding VARCHAR(255),
    platform_code VARCHAR(255),
    geometry public.GEOMETRY(POINT, 4326)
);
"""))

stops_data.to_sql('stops', conn, if_exists='append', index=False, dtype={'geometry': Geometry('POINT', srid)})

# Create indexes for the stops table
conn.execute(text("""CREATE INDEX IF NOT EXISTS stop_id ON Stops("stop_id");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS stop_location ON Stops USING gist(geometry);"""))

query(conn, "SELECT * FROM Stops;")

/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/98570548.py:13: UserWarning: Geometry column does not contain geometry.
  stops_data['geometry'] = stops_data['geometry'].apply(create_point_wkt_element)


,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code,geometry
0,200039,200039.0,"Central Station, Eddy Av, Stand A",-33.8822064874687,151.20666465471,None,200060,0,None,0101000020E6100000FFA631FF9CE66240A1FF6524ECF0...
1,200054,200054.0,"Central Station, Eddy Av, Stand D",-33.8820421431408,151.20699145565,None,200060,0,None,0101000020E61000002F928BAC9FE66240E33DC7C1E6F0...
2,200060,None,Central Station,-33.8840842535493,151.206292455081,1.0,None,0,None,0101000020E6100000817FA2F299E662408FF33DAC29F1...
3,201510,None,Redfern Station,-33.8916900512711,151.198866071817,1.0,None,0,None,0101000020E61000009E57611C5DE6624060304CE622F2...
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",-33.8933293130144,151.198881722942,None,201510,0,None,0101000020E6100000DBF9333D5DE662403DFA6B9D58F2...
...,...,...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",-33.8220164586429,151.07879697831,None,21271,1,B,0101000020E6100000AF9B3D8185E262408F52D7D537E9...
114714,2137185,2137185.0,"Cabarita Wharf, Side A",-33.8406690716775,151.116926480557,None,21371,1,1A,0101000020E6100000EB409ADCBDE3624089CE4C0B9BEB...
114715,2137186,2137186.0,"Cabarita Wharf, Side B",-33.8407691073139,151.116898892402,None,21371,1,1B,0101000020E6100000C4F9BEA2BDE362403EB375529EEB...
114716,21501,21501.0,Parramatta Wharf,-33.8139042429414,151.010576673346,None,2150112,1,None,0101000020E6100000E443E4A456E0624025C1A4032EE8...


In [10]:
#Load the polling places data
polling_places = pd.read_csv('PollingPlaces2019.csv')
polling_places = polling_places[polling_places['latitude'].notna()]

polling_places['the_geom'] = gpd.points_from_xy(polling_places.longitude, polling_places.latitude)

polling_places['geometry'] = polling_places['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
polling_places = polling_places.drop(columns=['longitude', 'latitude', 'the_geom'])

conn.execute(text("""
DROP TABLE IF EXISTS Poll;
CREATE TABLE Poll(
    "FID" VARCHAR(255),
    "state" VARCHAR(255),
    "division_id" VARCHAR(255),
    "division_name" VARCHAR(255),
    "polling_place_id" VARCHAR(255),
    "polling_place_type_id" VARCHAR(255),
    "polling_place_name" VARCHAR(255),
    "premises_name" VARCHAR(255),
    "premises_address_1" VARCHAR(255),
    "premises_address_2" VARCHAR(255),
    "premises_address_3" VARCHAR(255),
    "premises_suburb" VARCHAR(255),
    "premises_state_abbreviation" VARCHAR(255),
    "premises_post_code" VARCHAR(255),
    geometry public.GEOMETRY(POINT, 4326)
);
"""))

polling_places.to_sql('poll', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('POINT', srid)})

# Create indexes for the poll table
conn.execute(text("""CREATE INDEX IF NOT EXISTS polling_place_id ON Poll("polling_place_id");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS polling_location ON Poll USING gist(geometry);"""))

query(conn, "SELECT * FROM Poll;")

,FID,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,geometry
0,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,58,1,Oatley,Oatley Public School,51 Letitia St,None,None,OATLEY,NSW,2223.0,0101000020E61000003BDF4F8D97E26240832F4CA60AFE...
1,aec_federal_election_polling_places_2019.fid-4...,NSW,111,Chifley,392,1,Dharruk,Dawson Public School,7 Stuart Rd,None,None,DHARRUK,NSW,2770.0,0101000020E6100000A01A2FDD24DA624048E17A14AEDF...
2,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,31,1,Allawah,PJ Ferry Reserve Community Hall,147B Bellevue Pde,None,None,ALLAWAH,NSW,2218.0,0101000020E6100000A7EC4F3DADE36240EA48E47107FD...
3,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,67,1,Allawah South,St Raphael's Church Hall,84 George St,None,None,SOUTH HURSTVILLE,NSW,2221.0,0101000020E6100000643BDF4F8DE3624022FDF675E0FC...
4,aec_federal_election_polling_places_2019.fid-4...,NSW,103,Banks,56500,1,Beverly Hills North (Banks),Beverly Hills North Public School,1-3 Shorter Ave,None,None,BEVERLY HILLS,NSW,2209.0,0101000020E61000006666666666E26240C6DCB5847CF8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2785,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2810,1,Warilla North,Warilla North Community Centre,2-6 Hill St,None,None,WARILLA,NSW,2528.0,0101000020E6100000D8B628B341DB62401288D7F50B46...
2786,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,None,None,BARRACK HEIGHTS,NSW,2528.0,0101000020E6100000FA7E6ABC74DB62409C33A2B43748...
2787,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,None,None,None,WARILLA,NSW,2528.0,0101000020E6100000BD32141C83DB624011F28B5C8146...
2788,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,None,None,WELBY,NSW,2575.0,0101000020E610000021B0726891CD6240386744696F38...


In [11]:
#Load the catchment data
catchments_primary = gpd.read_file("catchments/catchments_primary.shp")
catchments_future = gpd.read_file("catchments/catchments_future.shp")
catchments_secondary = gpd.read_file("catchments/catchments_secondary.shp")

def create_catchment_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

catchments_primary = catchments_primary[catchments_primary['geometry'].notna()]
catchments_primary['geometry'] = catchments_primary['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))

catchments_future = catchments_future[catchments_future['geometry'].notna()]
catchments_future['geometry'] = catchments_future['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))

catchments_secondary = catchments_secondary[catchments_secondary['geometry'].notna()]
catchments_secondary['geometry'] = catchments_secondary['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))

conn.execute("""
DROP TABLE IF EXISTS catchments_primary;
CREATE TABLE catchments_primary(
    "USE_ID" INTEGER,
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" INTEGER,
    "KINDERGART" VARCHAR(255),
    "YEAR1" VARCHAR(255),
    "YEAR2" VARCHAR(255),
    "YEAR3" VARCHAR(255),
    "YEAR4" VARCHAR(255),
    "YEAR5" VARCHAR(255),
    "YEAR6" VARCHAR(255),
    "YEAR7" VARCHAR(255),
    "YEAR8" VARCHAR(255),
    "YEAR9" VARCHAR(255),
    "YEAR10" VARCHAR(255),
    "YEAR11" VARCHAR(255),
    "YEAR12" VARCHAR(255),
    "PRIORITY" VARCHAR(255),
    geometry public.GEOMETRY(MULTIPOLYGON, 4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS catchments_future;
CREATE TABLE catchments_future(
    "USE_ID" INTEGER,
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" INTEGER,
    "KINDERGART" VARCHAR(255),
    "YEAR1" VARCHAR(255),
    "YEAR2" VARCHAR(255),
    "YEAR3" VARCHAR(255),
    "YEAR4" VARCHAR(255),
    "YEAR5" VARCHAR(255),
    "YEAR6" VARCHAR(255),
    "YEAR7" VARCHAR(255),
    "YEAR8" VARCHAR(255),
    "YEAR9" VARCHAR(255),
    "YEAR10" VARCHAR(255),
    "YEAR11" VARCHAR(255),
    "YEAR12" VARCHAR(255),
    "PRIORITY" VARCHAR(255),
    geometry public.GEOMETRY(MULTIPOLYGON, 4326)
);
""")

conn.execute("""
DROP TABLE IF EXISTS catchments_secondary;
CREATE TABLE catchments_secondary(
    "USE_ID" INTEGER,
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" INTEGER,
    "KINDERGART" VARCHAR(255),
    "YEAR1" VARCHAR(255),
    "YEAR2" VARCHAR(255),
    "YEAR3" VARCHAR(255),
    "YEAR4" VARCHAR(255),
    "YEAR5" VARCHAR(255),
    "YEAR6" VARCHAR(255),
    "YEAR7" VARCHAR(255),
    "YEAR8" VARCHAR(255),
    "YEAR9" VARCHAR(255),
    "YEAR10" VARCHAR(255),
    "YEAR11" VARCHAR(255),
    "YEAR12" VARCHAR(255),
    "PRIORITY" VARCHAR(255),
    geometry public.GEOMETRY(MULTIPOLYGON, 4326)
);
""")
school_catchments = pd.concat([catchments_primary, catchments_future, catchments_secondary])
catchments_primary.to_sql('catchments_primary', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid)})
catchments_future.to_sql('catchments_future', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid)})
catchments_secondary.to_sql('catchments_secondary', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid)})

# Create indexes for the catchments tables
conn.execute(text("""CREATE INDEX IF NOT EXISTS idx_catchments_primary_use_id ON catchments_primary("USE_ID");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS idx_catchments_future_use_id ON catchments_future("USE_ID");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS idx_catchments_secondary_use_id ON catchments_secondary("USE_ID");"""))

conn.execute("""
DROP TABLE IF EXISTS school_catchments;
CREATE TABLE school_catchments(
    "USE_ID" INTEGER,
    "CATCH_TYPE" VARCHAR(255),
    "USE_DESC" VARCHAR(255),
    "ADD_DATE" INTEGER,
    "KINDERGART" VARCHAR(255),
    "YEAR1" VARCHAR(255),
    "YEAR2" VARCHAR(255),
    "YEAR3" VARCHAR(255),
    "YEAR4" VARCHAR(255),
    "YEAR5" VARCHAR(255),
    "YEAR6" VARCHAR(255),
    "YEAR7" VARCHAR(255),
    "YEAR8" VARCHAR(255),
    "YEAR9" VARCHAR(255),
    "YEAR10" VARCHAR(255),
    "YEAR11" VARCHAR(255),
    "YEAR12" VARCHAR(255),
    "PRIORITY" VARCHAR(255),
    geometry public.GEOMETRY(MULTIPOLYGON, 4326)
);
""")

school_catchments = pd.concat([catchments_primary, catchments_future, catchments_secondary])
catchments_primary.to_sql('school_catchments', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid)})

conn.execute(text("""CREATE INDEX IF NOT EXISTS school_catchments_ID ON school_catchments("USE_ID");"""))
conn.execute(text("""CREATE INDEX IF NOT EXISTS school_catchments_location ON school_catchments USING gist(geometry)"""))

query(conn, "SELECT * FROM school_catchments;")

/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/3574180327.py:12: UserWarning: Geometry column does not contain geometry.
  catchments_primary['geometry'] = catchments_primary['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))
/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/3574180327.py:15: UserWarning: Geometry column does not contain geometry.
  catchments_future['geometry'] = catchments_future['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))
/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/3574180327.py:18: UserWarning: Geometry column does not contain geometry.
  catchments_secondary['geometry'] = catchments_secondary['geometry'].apply(lambda x: create_catchment_wkt_element(geom=x,srid=srid))


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000078...
1,2404,PRIMARY,Lindfield EPS,20211219.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000BE...
2,4393,PRIMARY,Carlingford WPS,20220223.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000065...
3,7308,PRIMARY,Plattsburg PS,20200723.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E6100000010000000103000000010000003D...
4,4615,PRIMARY,Caddies Ck PS,20181210.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E61000000100000001030000000100000056...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,2409,PRIMARY,Lismore SPS,20200616.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000C9...
1658,4383,PRIMARY,E A Southee PS,20200315.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000B9...
1659,3275,PRIMARY,Tumbarumba PS,20200507.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000C0...
1660,2239,PRIMARY,Jindera PS,20200507.0,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,0106000020E610000001000000010300000001000000F6...


In [12]:
#Task2 Compute the score
inspect(db).get_table_names(schema='public')

['spatial_ref_sys',
 'bicycle_parking',
 'bicycle_parking_region',
 'car_share_bay',
 'car_share_bay_region',
 'Bike_lockers_sheds',
 'business_per_1000',
 'stops_number_region',
 'polls_number_region',
 'school_catchments_region',
 'area_score',
 'area_z_score',
 'bike_lockers_sheds',
 'bike_lockers_sheds_region',
 'petrol_stations',
 'petrol_stations_region',
 'traffic_lights_location',
 'traffic_lights_region',
 'area_score_total',
 'area_z_score_total',
 'correlation',
 'visualise',
 'businesses',
 'sa2',
 'population',
 'income',
 'stops',
 'poll',
 'catchments_primary',
 'catchments_future',
 'catchments_secondary',
 'school_catchments']

In [13]:
#Get the Business per 1000 people Z-score table
sql_business = """
DROP TABLE IF EXISTS Business_per_1000;
CREATE TABLE Business_per_1000 AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    business_temporary."business_count",
    (business_temporary."business_count")::FLOAT/Population."total_people"*1000 AS "business_per_1000",
    AVG((business_temporary."business_count")::FLOAT/Population."total_people"*1000) OVER() AS "mean",
    STDDEV_POP((business_temporary."business_count")::FLOAT/Population."total_people"*1000) OVER() AS "sd"
FROM
    Population
LEFT JOIN(
    SELECT
        "sa2_code",
        SUM("total_businesses") AS "business_count"
    FROM
        Businesses
    GROUP BY 
        "sa2_code"
) AS business_temporary
ON 
    Population."sa2_code" = business_temporary."sa2_code"
WHERE
    Population."total_people" > 100;
"""
conn.execute(text(sql_business))
query(conn, "SELECT * FROM Business_per_1000")

,sa2_code,sa2_name,business_count,business_per_1000,mean,sd
0,102011028,Avoca Beach - Copacabana,798,105.976096,125.081869,247.413412
1,102011029,Box Head - MacMasters Beach,993,89.847991,125.081869,247.413412
2,102011030,Calga - Kulnura,1218,256.529065,125.081869,247.413412
3,102011031,Erina - Green Point,1844,124.569344,125.081869,247.413412
4,102011032,Gosford - Springfield,2435,114.072894,125.081869,247.413412
...,...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,1549,69.929123,125.081869,247.413412
356,128021538,Sutherland - Kirrawee,2205,94.355770,125.081869,247.413412
357,128021607,Engadine,1095,63.007078,125.081869,247.413412
358,128021608,Loftus - Yarrawarrah,437,59.423443,125.081869,247.413412


In [14]:
#Get a table having the number of stops per region
sql_stops = """
DROP TABLE IF EXISTS Stops_number_region;
CREATE TABLE Stops_number_region AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    COUNT(Stops."stop_id") AS "stop_quantity",
    AVG(COUNT(Stops."stop_id")) OVER() AS "mean",
    STDDEV_POP(COUNT(Stops."stop_id")) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    Stops
ON
    ST_Contains(sa2.geometry, Stops.geometry)
RIGHT JOIN
    Population
ON
    sa2."SA2_CODE21" = Population."sa2_code"
WHERE
    Population."total_people" > 100
GROUP BY
    Population."sa2_code", Population."sa2_name";
"""
conn.execute(text(sql_stops))
query(conn, "SELECT * FROM Stops_number_region")

,sa2_code,sa2_name,stop_quantity,mean,sd
0,123011702,Spring Farm,47,153.441667,84.850666
1,102021049,Jilliby - Yarramalong,232,153.441667,84.850666
2,120031678,Burwood (NSW),74,153.441667,84.850666
3,115041301,Pitt Town - McGraths Hill,268,153.441667,84.850666
4,126021498,Gladesville - Huntleys Point,134,153.441667,84.850666
...,...,...,...,...,...
355,126021503,West Ryde - Meadowbank,160,153.441667,84.850666
356,127011505,Austral - Greendale,206,153.441667,84.850666
357,119041381,Rockdale - Banksia,122,153.441667,84.850666
358,125041719,Rosehill - Harris Park,59,153.441667,84.850666


In [15]:
# Get a table having the number of polls per region
sql_poll = """
DROP TABLE IF EXISTS Polls_number_region;
CREATE TABLE Polls_number_region AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    COUNT(Poll.polling_place_id) AS "polling_quantity",
    AVG(COUNT(Poll.polling_place_id)) OVER() AS "mean",
    STDDEV_POP(COUNT(Poll.polling_place_id)) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    Poll
ON
    ST_Contains(sa2.geometry, Poll.geometry)
RIGHT JOIN
    Population
ON
    sa2."SA2_CODE21" = Population."sa2_code"
WHERE
    Population."total_people" > 100
GROUP BY
    Population."sa2_code", Population."sa2_name";
"""
conn.execute(text(sql_poll))

query(conn, "SELECT * FROM Polls_number_region")

,sa2_code,sa2_name,polling_quantity,mean,sd
0,102011028,Avoca Beach - Copacabana,2,4.213889,4.123284
1,102011029,Box Head - MacMasters Beach,2,4.213889,4.123284
2,102011030,Calga - Kulnura,7,4.213889,4.123284
3,102011031,Erina - Green Point,6,4.213889,4.123284
4,102011032,Gosford - Springfield,7,4.213889,4.123284
...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,7,4.213889,4.123284
356,128021538,Sutherland - Kirrawee,6,4.213889,4.123284
357,128021607,Engadine,7,4.213889,4.123284
358,128021608,Loftus - Yarrawarrah,2,4.213889,4.123284


In [16]:
#Get a table having the school data
sql_school_catchments = """
DROP TABLE IF EXISTS school_catchments_region;
CREATE TABLE school_catchments_region AS
SELECT
    Youth_population."sa2_code",
    Youth_population."sa2_name",
    COUNT(school_catchments."USE_ID") AS "catchments_quantity",
    COUNT(school_catchments."USE_ID")::FLOAT/Youth_population."youth"*1000 AS "school_catchments_per_1000_people",
    AVG(COUNT(school_catchments."USE_ID")::FLOAT/Youth_population."youth"*1000) OVER() AS "mean",
    STDDEV_POP(COUNT(school_catchments."USE_ID")::FLOAT/Youth_population."youth"*1000) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    school_catchments
ON
    ST_Intersects(sa2.geometry, school_catchments.geometry)
RIGHT JOIN
    (SELECT *, "0-4_people"+"5-9_people"+"10-14_people"+"15-19_people" AS "youth"
        FROM
    Population) AS Youth_population
ON
    Youth_population."sa2_code" = sa2."SA2_CODE21"
WHERE
    Youth_population."total_people">100
GROUP BY
    Youth_population."sa2_code", Youth_population."sa2_name", Youth_population."youth";
"""
conn.execute(text(sql_school_catchments))
query(conn, "SELECT * FROM school_catchments_region")

,sa2_code,sa2_name,catchments_quantity,school_catchments_per_1000_people,mean,sd
0,102011028,Avoca Beach - Copacabana,4,1.885903,2.578062,3.978277
1,102011029,Box Head - MacMasters Beach,4,1.618778,2.578062,3.978277
2,102011030,Calga - Kulnura,17,17.689906,2.578062,3.978277
3,102011031,Erina - Green Point,10,3.120125,2.578062,3.978277
4,102011032,Gosford - Springfield,10,2.291476,2.578062,3.978277
...,...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,9,1.448809,2.578062,3.978277
356,128021538,Sutherland - Kirrawee,12,2.363135,2.578062,3.978277
357,128021607,Engadine,7,1.367722,2.578062,3.978277
358,128021608,Loftus - Yarrawarrah,6,2.894356,2.578062,3.978277


In [17]:
query(conn, "SELECT * FROM school_catchments_region")

,sa2_code,sa2_name,catchments_quantity,school_catchments_per_1000_people,mean,sd
0,102011028,Avoca Beach - Copacabana,4,1.885903,2.578062,3.978277
1,102011029,Box Head - MacMasters Beach,4,1.618778,2.578062,3.978277
2,102011030,Calga - Kulnura,17,17.689906,2.578062,3.978277
3,102011031,Erina - Green Point,10,3.120125,2.578062,3.978277
4,102011032,Gosford - Springfield,10,2.291476,2.578062,3.978277
...,...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,9,1.448809,2.578062,3.978277
356,128021538,Sutherland - Kirrawee,12,2.363135,2.578062,3.978277
357,128021607,Engadine,7,1.367722,2.578062,3.978277
358,128021608,Loftus - Yarrawarrah,6,2.894356,2.578062,3.978277


In [18]:
# Compute the Z-score and Total score for each area
sql_score = """
DROP TABLE IF EXISTS area_score;
CREATE TABLE area_score AS
SELECT
    B."sa2_code",
    B."sa2_name",
    (B."business_per_1000" - B."mean")/B."sd" AS "Business_Z-Score",
    (S."stop_quantity" - S."mean")/S."sd" AS "Stops_number_region_Z-Score",
    (P."polling_quantity" - P."mean")/P."sd" AS "Polls_number_region_Z-Score",
    (C."school_catchments_per_1000_people" - C."mean")/C."sd" AS "school_catchments_region_Z-Score"
FROM 
    Business_per_1000 B, 
    Stops_number_region S, 
    Polls_number_region P, 
    school_catchments_region C
WHERE
    B."sa2_code" = S."sa2_code"
    AND
    S."sa2_code" = P."sa2_code"
    AND
    P."sa2_code" = C."sa2_code";


DROP TABLE IF EXISTS area_Z_Score;
CREATE TABLE area_Z_Score AS
SELECT
    *,
    (1/(1 + EXP(-("Business_Z-Score"+"Stops_number_region_Z-Score"+"Polls_number_region_Z-Score"+"school_catchments_region_Z-Score")))) AS "Score"
FROM
    area_score;
"""
conn.execute(text(sql_score))

query(conn, "SELECT * FROM area_Z_score")

,sa2_code,sa2_name,Business_Z-Score,Stops_number_region_Z-Score,Polls_number_region_Z-Score,school_catchments_region_Z-Score,Score
0,102011028,Avoca Beach - Copacabana,-0.077222,-0.311626,-0.536924,-0.173985,0.249786
1,102011029,Box Head - MacMasters Beach,-0.142409,0.725490,-0.536924,-0.241130,0.451411
2,102011030,Calga - Kulnura,0.531286,0.218718,0.675702,3.798590,0.994645
3,102011031,Erina - Green Point,-0.002072,1.550469,0.433177,0.136256,0.892624
4,102011032,Gosford - Springfield,-0.044496,2.375448,0.675702,-0.072038,0.949531
...,...,...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,-0.222917,1.185121,0.675702,-0.283855,0.794791
356,128021538,Sutherland - Kirrawee,-0.124189,0.489782,0.433177,-0.054025,0.678032
357,128021607,Engadine,-0.250895,0.796203,0.675702,-0.304237,0.714384
358,128021608,Loftus - Yarrawarrah,-0.265379,-0.111274,-0.536924,0.079505,0.302785


In [19]:
score = query(conn, "SELECT * FROM area_Z_score")

In [20]:
#Find the minimum score city's name
min_score_index = score['Score'].idxmin()
area_with_min_score = score.loc[min_score_index, 'sa2_name']
print(area_with_min_score)

Spring Farm


In [21]:
#Find the maximum score city' name
max_score_index = score['Score'].idxmax()
area_with_max_score = score.loc[max_score_index, 'sa2_name']
print(area_with_max_score)

Sydney (North) - Millers Point


In [22]:
#Task 3

In [23]:
#Count how many bicycle parking site does each region have

#Load the data which contains the bicycle parking sites

bike_lockers_sheds_file = pd.read_csv('bike_lockers_sheds.csv')
bike_lockers_sheds_file = bike_lockers_sheds_file[bike_lockers_sheds_file['Latitude'].notna()]

bike_lockers_sheds_file['the_geom'] = gpd.points_from_xy(bike_lockers_sheds_file.Longitude, bike_lockers_sheds_file.Latitude)

bike_lockers_sheds_file['geometry'] = bike_lockers_sheds_file['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))


bike_lockers_sheds = bike_lockers_sheds_file[['Location', 'geometry']]

conn.execute("""
DROP TABLE IF EXISTS Bike_lockers_sheds;
CREATE TABLE Bike_lockers_sheds(
    "Location" VARCHAR(255),
    geometry public.GEOMETRY(POINT, 4326)
)
""")

bike_lockers_sheds.to_sql('bike_lockers_sheds', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('POINT', srid)})


# Get a table having the number of bicycle_parking per region
sql_Bike_lockers_sheds = """
DROP TABLE IF EXISTS bike_lockers_sheds_region;
CREATE TABLE bike_lockers_sheds_region AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    COUNT(Bike_lockers_sheds."Location") AS "sheds_quantity",
    AVG(COUNT(Bike_lockers_sheds."Location")) OVER() AS "mean",
    STDDEV_POP(COUNT(Bike_lockers_sheds."Location")) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    Bike_lockers_sheds
ON
    ST_Contains(sa2.geometry, Bike_lockers_sheds.geometry)
RIGHT JOIN
    Population
ON
    sa2."SA2_CODE21" = Population."sa2_code"
WHERE
    Population."total_people" > 100
GROUP BY
    Population."sa2_code", Population."sa2_name";
"""
conn.execute(text(sql_Bike_lockers_sheds))
query(conn, "SELECT * FROM bike_lockers_sheds_region")

,sa2_code,sa2_name,sheds_quantity,mean,sd
0,102011028,Avoca Beach - Copacabana,0,0.422222,0.829696
1,102011029,Box Head - MacMasters Beach,0,0.422222,0.829696
2,102011030,Calga - Kulnura,0,0.422222,0.829696
3,102011031,Erina - Green Point,0,0.422222,0.829696
4,102011032,Gosford - Springfield,2,0.422222,0.829696
...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,2,0.422222,0.829696
356,128021538,Sutherland - Kirrawee,3,0.422222,0.829696
357,128021607,Engadine,1,0.422222,0.829696
358,128021608,Loftus - Yarrawarrah,1,0.422222,0.829696


In [24]:
#Count how many Petrol Stations does each region have

#Load the data which contains the Petrol Stations
petrol_stations_file = gpd.read_file("Petrol_Stations.geojson")
petrol_stations = petrol_stations_file[['objectid', 'geometry']]

petrol_stations.loc[:, 'geometry'] = petrol_stations['geometry'].apply(lambda x: WKTElement(x.wkt, srid=srid))

conn.execute("""
DROP TABLE IF EXISTS Petrol_stations;
CREATE TABLE Petrol_stations(
    "objectid" VARCHAR(50),
    geometry public.GEOMETRY(POINT, 4326)
)
""")

petrol_stations.to_sql('petrol_stations', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('POINT', srid)})


# Get a table having the number of car_share_bay per region
sql_petrol_stations = """
DROP TABLE IF EXISTS petrol_stations_region;
CREATE TABLE petrol_stations_region AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    COUNT(Petrol_stations."objectid") AS "petrol_stations_quantity",
    AVG(COUNT(Petrol_stations."objectid")) OVER() AS "mean",
    STDDEV_POP(COUNT(Petrol_stations."objectid")) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    petrol_stations
ON
    ST_Contains(sa2.geometry, Petrol_stations.geometry)
RIGHT JOIN
    Population
ON
    sa2."SA2_CODE21" = Population."sa2_code"
WHERE
    Population."total_people" > 100
GROUP BY
    Population."sa2_code", Population."sa2_name";
"""
conn.execute(text(sql_petrol_stations))
query(conn, "SELECT * FROM petrol_stations_region")

/var/folders/3q/58khkx012qxb5lsgts3s5f0r0000gn/T/ipykernel_88848/1848150910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  petrol_stations.loc[:, 'geometry'] = petrol_stations['geometry'].apply(lambda x: WKTElement(x.wkt, srid=srid))


,sa2_code,sa2_name,petrol_stations_quantity,mean,sd
0,102011028,Avoca Beach - Copacabana,1,2.597222,2.101981
1,102011029,Box Head - MacMasters Beach,1,2.597222,2.101981
2,102011030,Calga - Kulnura,4,2.597222,2.101981
3,102011031,Erina - Green Point,4,2.597222,2.101981
4,102011032,Gosford - Springfield,5,2.597222,2.101981
...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,2,2.597222,2.101981
356,128021538,Sutherland - Kirrawee,6,2.597222,2.101981
357,128021607,Engadine,2,2.597222,2.101981
358,128021608,Loftus - Yarrawarrah,1,2.597222,2.101981


In [25]:
#Count how many traffic_lights_location does each region have

#Load the data which contains the traffic_lights_location

traffic_lights_location_file = pd.read_csv('traffic_lights_location.csv')
traffic_lights_location_file = traffic_lights_location_file[traffic_lights_location_file['latitude'].notna()]

traffic_lights_location_file['the_geom'] = gpd.points_from_xy(traffic_lights_location_file.longitude, traffic_lights_location_file.latitude)

traffic_lights_location_file['geometry'] = traffic_lights_location_file['the_geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))


traffic_lights = traffic_lights_location_file[['Equipment_ID', 'geometry']]

conn.execute("""
DROP TABLE IF EXISTS Traffic_lights_location;
CREATE TABLE Traffic_lights_location(
    "Equipment_ID" VARCHAR(255),
    geometry public.GEOMETRY(POINT, 4326)
)
""")

traffic_lights.to_sql('traffic_lights_location', conn, if_exists = 'append', index=False, dtype={'geometry': Geometry('POINT', srid)})


# Get a table having the number of bicycle_parking per region
sql_traffic_lights = """
DROP TABLE IF EXISTS traffic_lights_region;
CREATE TABLE traffic_lights_region AS
SELECT
    Population."sa2_code",
    Population."sa2_name",
    COUNT(Traffic_lights_location."Equipment_ID") AS "traffic_lights_quantity",
    AVG(COUNT(Traffic_lights_location."Equipment_ID")) OVER() AS "mean",
    STDDEV_POP(COUNT(Traffic_lights_location."Equipment_ID")) OVER() AS "sd"
FROM
    sa2
LEFT JOIN
    Traffic_lights_location
ON
    ST_Contains(sa2.geometry, Traffic_lights_location.geometry)
RIGHT JOIN
    Population
ON
    sa2."SA2_CODE21" = Population."sa2_code"
WHERE
    Population."total_people" > 100
GROUP BY
    Population."sa2_code", Population."sa2_name";
"""
conn.execute(text(sql_traffic_lights))



query(conn, "SELECT * FROM traffic_lights_region")

,sa2_code,sa2_name,traffic_lights_quantity,mean,sd
0,102011028,Avoca Beach - Copacabana,0,9.466667,8.416518
1,102011029,Box Head - MacMasters Beach,2,9.466667,8.416518
2,102011030,Calga - Kulnura,6,9.466667,8.416518
3,102011031,Erina - Green Point,17,9.466667,8.416518
4,102011032,Gosford - Springfield,26,9.466667,8.416518
...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,4,9.466667,8.416518
356,128021538,Sutherland - Kirrawee,16,9.466667,8.416518
357,128021607,Engadine,2,9.466667,8.416518
358,128021608,Loftus - Yarrawarrah,1,9.466667,8.416518


In [26]:
# Compute the Z-score and Total score for each area
sql_score = """
DROP TABLE IF EXISTS area_score_total;
CREATE TABLE area_score_total AS
SELECT
    B."sa2_code",
    B."sa2_name",
    A."Business_Z-Score",
    A."Stops_number_region_Z-Score",
    A."Polls_number_region_Z-Score",
    A."school_catchments_region_Z-Score",
    (B."sheds_quantity" - B."mean") / B."sd" AS "bicycle_sheds_quantity_Z-Score",
    (P."petrol_stations_quantity" - P."mean") / P."sd" AS "petrol_stations_region_Z-Score",
    (T."traffic_lights_quantity" - T."mean") / T."sd" AS "traffic_lights_region_Z-Score"
FROM 
    bike_lockers_sheds_region B
JOIN 
    petrol_stations_region P ON B."sa2_code" = P."sa2_code"
JOIN 
    traffic_lights_region T ON B."sa2_code" = T."sa2_code"
JOIN 
    area_Z_Score A ON A.sa2_code = B.sa2_code;
    

DROP TABLE IF EXISTS area_Z_Score_total;
CREATE TABLE area_Z_Score_total AS
SELECT
    *,
    (1/(1 + EXP(-("Business_Z-Score"+"Stops_number_region_Z-Score"+"Polls_number_region_Z-Score"+"school_catchments_region_Z-Score"+"bicycle_sheds_quantity_Z-Score"+"petrol_stations_region_Z-Score"+"traffic_lights_region_Z-Score")))) AS "Score"
FROM
    area_score_total;
"""

conn.execute(text(sql_score))

query(conn, "SELECT * FROM area_Z_Score_total")

,sa2_code,sa2_name,Business_Z-Score,Stops_number_region_Z-Score,Polls_number_region_Z-Score,school_catchments_region_Z-Score,bicycle_sheds_quantity_Z-Score,petrol_stations_region_Z-Score,traffic_lights_region_Z-Score,Score
0,102011028,Avoca Beach - Copacabana,-0.077222,-0.311626,-0.536924,-0.173985,-0.508888,-0.759865,-1.124772,0.029504
1,102011029,Box Head - MacMasters Beach,-0.142409,0.725490,-0.536924,-0.241130,-0.508888,-0.759865,-0.887144,0.086997
2,102011030,Calga - Kulnura,0.531286,0.218718,0.675702,3.798590,-0.508888,0.667360,-0.411888,0.993111
3,102011031,Erina - Green Point,-0.002072,1.550469,0.433177,0.136256,-0.508888,0.667360,0.895065,0.959742
4,102011032,Gosford - Springfield,-0.044496,2.375448,0.675702,-0.072038,1.901634,1.143101,1.964391,0.999645
...,...,...,...,...,...,...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,-0.222917,1.185121,0.675702,-0.283855,1.901634,-0.284123,-0.649516,0.910686
356,128021538,Sutherland - Kirrawee,-0.124189,0.489782,0.433177,-0.054025,3.106895,1.618843,0.776251,0.998067
357,128021607,Engadine,-0.250895,0.796203,0.675702,-0.304237,0.696373,-0.284123,-0.887144,0.608706
358,128021608,Loftus - Yarrawarrah,-0.265379,-0.111274,-0.536924,0.079505,0.696373,-0.759865,-1.005958,0.129710


In [27]:
sql_score= """
DROP TABLE IF EXISTS correlation;
CREATE TABLE correlation AS
SELECT 
    a."sa2_code", 
    a."sa2_name", 
    a."Score", 
    i."median_income" 
FROM 
    area_Z_Score_total a
JOIN 
    income i
ON 
    a."sa2_code" = i."sa2_code21";
"""
conn.execute(text(sql_score))

query(conn, "SELECT * FROM correlation")

,sa2_code,sa2_name,Score,median_income
0,102011028,Avoca Beach - Copacabana,0.029504,55065
1,102011029,Box Head - MacMasters Beach,0.086997,51927
2,102011030,Calga - Kulnura,0.993111,49168
3,102011031,Erina - Green Point,0.959742,51905
4,102011032,Gosford - Springfield,0.999645,54372
...,...,...,...,...
355,128021536,Oyster Bay - Como - Jannali,0.910686,67381
356,128021538,Sutherland - Kirrawee,0.998067,64940
357,128021607,Engadine,0.608706,63695
358,128021608,Loftus - Yarrawarrah,0.129710,63087


In [28]:
#Visualisation
Visualise = """
DROP TABLE IF EXISTS Visualise;
CREATE TABLE Visualise AS
SELECT
    area_Z_Score_total.*,
    sa2."geometry" AS "geom"
FROM
    area_Z_Score_total
LEFT JOIN
    sa2
ON
    area_Z_Score_total."sa2_code" = sa2."SA2_CODE21"
"""
conn.execute(text(Visualise))


import io
import ipywidgets as widgets
from IPython.display import display, clear_output

# Function to create individual visualizations
def create_visualization(score_column, title):
    query = f"""
    SELECT
        area_Z_Score_total.*,
        sa2."geometry" AS "geom"
    FROM
        area_Z_Score_total
    LEFT JOIN
        sa2
    ON
        area_Z_Score_total."sa2_code" = sa2."SA2_CODE21"
    """
    conn.execute(text(query))
    
    visualise = gpd.read_postgis(f"SELECT * FROM Visualise", conn, crs=4326)
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    visualise.plot(column=score_column, cmap="viridis", legend=True, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    
    # Convert the plot to an image
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)  # Close the plot to avoid displaying it immediately
    return buf

# Create individual visualizations for each score
scores = ["Business_Z-Score", "Stops_number_region_Z-Score", "Polls_number_region_Z-Score",
          "school_catchments_region_Z-Score", "bicycle_sheds_quantity_Z-Score",
          "petrol_stations_region_Z-Score", "traffic_lights_region_Z-Score", "Score"]

titles = ["Business Z-Score", "Stops Number Region Z-Score", "Polls Number Region Z-Score",
          "School Catchments Region Z-Score", "Bicycle Sheds Quantity Z-Score",
          "Petrol Stations Region Z-Score", "Traffic Lights Region Z-Score", "Score"]

visualizations = [create_visualization(score, title) for score, title in zip(scores, titles)]

# Create a dropdown widget to select the visualization
dropdown = widgets.Dropdown(options=titles, description='Select Score:', disabled=False)

# Output widget to display the selected image
output = widgets.Output()

# Function to update the displayed image based on dropdown selection
def update_image(change):
    with output:
        clear_output(wait=True)  # Clear the previous output
        selected_index = titles.index(change.new)
        selected_visualization = visualizations[selected_index]
        selected_image = plt.imread(selected_visualization)
        plt.figure(figsize=(10, 10))
        plt.imshow(selected_image)
        plt.axis('off')
        plt.show()

dropdown.observe(update_image, names='value')

# Display the dropdown widget and output widget
display(dropdown)
display(output)


Dropdown(description='Select Score:', options=('Business Z-Score', 'Stops Number Region Z-Score', 'Polls Numbe…

Output()